In [1]:
import pickle
import numpy as np
import torch
import sys
sys.path.insert(0,'../src/')

with open("prune_model_dict.p", "rb") as handle:
    prune_dict = pickle.load(handle)
#with open("prune_gmp.p", "rb") as handle:
#    prune_gmp = pickle.load(handle)

In [ ]:
class compressed_model():
    def __init__(self, state_dict, gmp_list):
        #could be multiple gmps
        #could be multiple gmp scales
        gmp_means = []
        self.scale_len = 0
        for g in gmp_list:
            gmp_means.append(list(g.means.clone().data.cpu().numpy()))
            
        if (g.scaling):#if scaling , only one gmp should be present
            weights = torch.cat([state_dict[layer].view(-1) * g.scale.data.exp()[int(i/2)] for i,layer in enumerate(state_dict)]).cpu().numpy()
            self.scale_len = float(g.scale.size()[0])
        else:
            weights = torch.cat([state_dict[layer].view(-1) for layer in state_dict]).cpu().numpy()
            
        means = np.sort( np.append( np.array(gmp_means), np.zeros(1)) )
        bins = means + 0.001 * abs(means)
        binned_weights = np.digitize(pd, bins, right=True)

        unique, counts = np.unique(binned_weights, return_counts=True)
        zero_idx = np.argmax(counts)
        binned_weights[ binned_weights==unique[zero_idx] ] = 0
        new_means = []
        new_means.append(0.0)

        set_idx = 1
        for idx in unique:
            if idx != unique[zero_idx]:
                new_means.append(means[idx])
                binned_weights[ binned_weights==idx] = set_idx
                set_idx += 1
                
        self.binned_weights = binned_weights
        self.means = means

    def encode(self, index_bits):
        index_spacing = index_bits**2
        index_list = []
        weight_list = []
        index_counter=0
        for pos, weight in enumerate(list(self.binned_weights)):
            index_counter+=1
            if (weight==0):
                if index_counter==index_spacing:
                    index_list.append(index_spacing)
                    weight_list.append(0)
                    index_counter=0
            else:
                weight_list.append(weight)
                index_list.append(index_counter)
                index_counter=0
        if(index_counter>0):
            index_list.append(index_counter)
            weight_list.append(0)
        return index_list, weight_list
    
    def decode(self, index_list, weight_list):
        R_recov = []
        for index, weight in zip(index_list, weight_list):
            for z in range(int(index-1)):
                R_recov.append(0)
            R_recov.append(weight)
        return R_recov
    
    def recover(self):
        unique, counts = np.unique(self.binned_weights, return_counts=True)
        pd_recov = np.zeros(self.binned_weights.size)

        for u, nm in zip(unique, self.means):
            pd_recov[ binned_weights==u ] = nm
            print(u, nm)
            print(pd_recov)
        return pd_recov
    
    def get_cr(self, index_bits=0):
        full_size = self.binned_weights.size * 32
        codebook_size = 32 * (np.ceil(np.log2(self.means.size)) + np.ceil(np.log2(self.scale_len)))
        min_idx_bit = 0
        min_idx_size = -1
        if (index_bits !=0):
            index_list, weight_list = self.encode(index_bits)
            index_size = len(index_list) * index_bits
            weight_size = len(weight_list) * np.ceil(np.log2(self.means.size))
            return ((full_size / (codebook_size + weight_size + index_size)), min_idx_bit)
        
        for index_bits in range (6,11):
            index_list, weight_list = self.encode(index_bits)
            index_size = len(index_list) * index_bits
            weight_size = len(weight_list) * np.ceil(np.log2(self.means.size))
            if (index_size + weight_size < min_idx_size or min_idx_size ==-1):
                min_idx_size = index_size + weight_size
                min_idx_bit = index_bits
        return ((full_size / (codebook_size + min_idx_size)), min_idx_bit)
        
        
    
a = compressed_model(prune_dict, [prune_gmp])

In [ ]:
a.get_cr()

In [ ]:
a.means.size

In [4]:
from scipy.sparse import csc_matrix

csc_matrix()

In [15]:
csc_mat = csc_matrix(prune_dict['fc1.weight'])
A = csc_mat.data
IR = csc_mat.indices
IC = csc_mat.indptr

In [19]:
prune_dict['fc1.weight'].shape

torch.Size([500, 1250])

In [22]:
A.shape

(10204,)

In [21]:
csc_mat.indices.shape

(10204,)

In [20]:
csc_mat.indptr.shape

(1251,)

In [94]:
#model to unscaled means
pd = torch.cat([prune_dict[layer].view(-1) * prune_gmp.scale.data.exp()[int(i/2)] for i,layer in enumerate(prune_dict)]).cpu().numpy()

In [178]:
means = np.sort( np.append(prune_gmp.means.data.clone().cpu().numpy(), np.zeros(1)) )
bins = means + 0.001 * abs(means)
binned_weights = np.digitize(pd, bins, right=True)
print (binned_weights)

unique, counts = np.unique(binned_weights, return_counts=True)
print(dict(zip(unique, counts)))
print (unique)
print (counts)
zero_idx = np.argmax(counts)
binned_weights[ binned_weights==unique[zero_idx] ] = 0
print (binned_weights)
new_means = []
new_means.append(0.0)

set_idx = 1
for idx in unique:
    if idx != unique[zero_idx]:
        new_means.append(means[idx])
        binned_weights[ binned_weights==idx] = set_idx
        set_idx += 1

[ 7  8  9 ...  8 11  8]
{1: 2303, 3: 3448, 5: 3701, 7: 15, 8: 628409, 9: 660, 11: 2626, 13: 1050, 14: 241, 15: 7}
[ 1  3  5  7  8  9 11 13 14 15]
[  2303   3448   3701     15 628409    660   2626   1050    241      7]
[ 7  0  9 ...  0 11  0]


In [179]:
unique, counts = np.unique(binned_weights, return_counts=True)
pd_recov = np.zeros(binned_weights.size)

for u, nm in zip(unique, new_means):
    pd_recov[ binned_weights==u ] = nm
    print(u, nm)
    print(pd_recov)
pd_recov

0 0.0
[0. 0. 0. ... 0. 0. 0.]
1 -0.5548520684242249
[0. 0. 0. ... 0. 0. 0.]
2 -0.275088906288147
[0. 0. 0. ... 0. 0. 0.]
3 -0.16535350680351257
[0. 0. 0. ... 0. 0. 0.]
4 -0.04979413375258446
[-0.04979413  0.          0.         ...  0.          0.
  0.        ]
5 0.08361522108316422
[-0.04979413  0.          0.08361522 ...  0.          0.
  0.        ]
6 0.19607827067375183
[-0.04979413  0.          0.08361522 ...  0.          0.19607827
  0.        ]
7 0.3386446237564087
[-0.04979413  0.          0.08361522 ...  0.          0.19607827
  0.        ]
8 0.5209544897079468
[-0.04979413  0.          0.08361522 ...  0.          0.19607827
  0.        ]
9 0.7065592408180237
[-0.04979413  0.          0.08361522 ...  0.          0.19607827
  0.        ]


array([-0.04979413,  0.        ,  0.08361522, ...,  0.        ,
        0.19607827,  0.        ])

In [185]:
(pd - pd_recov).max()

0.0

In [ ]:
[[x,list(pd).count(x)] for x in set(list(pd))]

In [1]:
#naive calculation
nb = 200000 * 32
kb = 16 * 32
nl2k = 0.005 * 200000 * np.log2(16) + 200000/16 * np.log2(16)
(nb/(kb + nl2k))

NameError: name 'np' is not defined

In [2]:
number_of_elements = 266610
pruning_rate = 0.043/2
index_bits = 5
#number_of_elements = 400000
#pruning_rate = 0.005/2
#index_bits = 6

orig = number_of_elements * 32 #266k elements at 32 bit precision
codebook = 16 * 32 # 16 cluster means at 32 bit preicison
weight = number_of_elements * np.log2(32) * (pruning_rate + 1/(2**index_bits))
index = number_of_elements * index_bits * (pruning_rate + 1/(2**index_bits))
(orig/(codebook + weight + index))



60.443457621222713

In [187]:
#encode
R = np.random.rand(400000)
R[R<=0.995] = 0
R[R>0.995] = 1/100
'''
R[R>0.990] = 2/100
R[R>0.985] = 3/100
R[R>0.980] = 4/100
R[R>0.975] = 5/100
R[R>0.970] = 6/100
R[R>0.965] = 7/100
R[R>0.960] = 8/100
R[R>0.959] = 9/100
R[R>0.957] = 10/100
'''

R=R*100
#[[x,list(R).count(x)] for x in set(list(R))]

#code_list


print(full_size, len(weight_list), index_size, weight_size, codebook_size, full_size/(index_size + weight_size + codebook_size))

NameError: name 'index_list' is not defined

In [6]:
#decode


In [7]:
(R - R_recov).sum()

0.0